In [14]:
from huggingface_hub import login
login()

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,   # use float32 if no GPU
    device_map="auto",           # automatically uses GPU if available
    use_auth_token=True
)


/home/I6219533/finetune_example/ft_env/lib/python3.12/site-packages/transformers/models/auto/tokenization_auto.py:1041: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/I6219533/finetune_example/ft_env/lib/python3.12/site-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
messages = [
    {"role": "system", "content": "انت سوري تتكلم بالهجة السورية"},
    {"role": "user", "content": "Translate to Syrian English: قلي مابحسن اروح لهنيك "}
]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)


In [12]:

inputs = tokenizer(
    prompt,
    return_tensors="pt"
).to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        temperature=0.7,
        top_p=0.9,
        do_sample=True
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


system

Cutting Knowledge Date: December 2023
Today Date: 16 Dec 2025

انت سوري تتكلم بالهجة السوريةuser

Translate to Syrian English: قلي مابحسن اروح لهنيكassistant

Your translation is close, but I'll provide a more precise translation:

"Qoli mabahsin, irhouh leenak" 

Breakdown:

- قلي (Qoli) means "go" or "let's go".
- مابحسن (mabahsin) is a polite way of saying "please" or "you are welcome".
- اروح (irhouh) means "go" (again, in the imperative form).
- لهنيك (leeneak) is the accusative plural form of "hena", which means "you".

So, the more accurate translation is: "Please, let's go, you are welcome."


# FINE-TUNING

In [1]:
from datasets import load_dataset

# For a simple text file
dataset = load_dataset("text", data_files={"train": "syrian.txt"})

# Inspect
print(dataset["train"][0])


{'text': ' قامت على طاولتها واخدت الشنته وطلعت مع الكفتيريا '}


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "meta-llama/Llama-3.2-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",   # automatically use GPU
    torch_dtype=torch.float16,
    use_auth_token=True
)


/home/I6219533/finetune_example/ft_env/lib/python3.12/site-packages/transformers/models/auto/tokenization_auto.py:1041: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/I6219533/finetune_example/ft_env/lib/python3.12/site-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
def tokenize_and_add_labels(examples):
    tokenized = tokenizer(examples["text"], truncation=True, max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()  # for causal LM
    return tokenized

tokenized_dataset = dataset.map(tokenize_and_add_labels, batched=True)


In [4]:
from peft import LoraConfig, get_peft_model

# Configure LoRA
lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["q_proj","v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


In [5]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./syrian_dialect_lora_test3",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=5,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    save_total_limit=2,
    report_to="none",
    save_safetensors=False  # << disable safetensors
)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"]
)

trainer.train(resume_from_checkpoint=True)


TrainOutput(global_step=31420, training_loss=0.56816256852153, metrics={'train_runtime': 4788.6849, 'train_samples_per_second': 26.245, 'train_steps_per_second': 6.561, 'total_flos': 6.605172768577536e+16, 'train_loss': 0.56816256852153, 'epoch': 5.0})

In [6]:
model.save_pretrained("./syrian_dialect_lora_test3")
tokenizer.save_pretrained("./syrian_dialect_lora_test3")


('./syrian_dialect_lora_test3/tokenizer_config.json',
 './syrian_dialect_lora_test3/special_tokens_map.json',
 './syrian_dialect_lora_test3/chat_template.jinja',
 './syrian_dialect_lora_test3/tokenizer.json')

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base_model_id = "meta-llama/Llama-3.2-3B-Instruct"
lora_path = "./syrian_dialect_lora_test3"
merged_output_path = "./shamidataset_finetunedmodel"

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Load LoRA adapter
model = PeftModel.from_pretrained(base_model, lora_path)

# Merge LoRA weights into base model
model = model.merge_and_unload()

# Save merged model
model.save_pretrained(merged_output_path, safe_serialization=True)

# Save tokenizer
tokenizer = AutoTokenizer.from_pretrained(lora_path)
tokenizer.save_pretrained(merged_output_path)


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

('./shamidataset_finetunedmodel/tokenizer_config.json',
 './shamidataset_finetunedmodel/special_tokens_map.json',
 './shamidataset_finetunedmodel/chat_template.jinja',
 './shamidataset_finetunedmodel/tokenizer.json')